https://adventofcode.com/2022/day/7

In [3]:
with open("data/07.txt")as fh:
    data = fh.read()

In [80]:
class Directory:
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.subdirs = {}
        self.files = {}
        self.subtree_size = 0

    def cd(self, subdir):
        if subdir == "..":
            if self.parent is None:
                raise IndexError("Can't 'cd ..' from root")
            return self.parent
        return self.subdirs[subdir]

    def add_subdir(self, name):
        if name in self.subdirs:
            raise KeyError(f"subdir {name} already exists")
        self.subdirs[name] = Directory(name, self)

    def add_file(self, name, size):
        if name in self.files:
            raise KeyError(f"file {name} already exists")
        self.files[name] = File(name, size)

    @property
    def size(self):
        return sum(x.size for x in self.files.values())

    def __repr__(self):
        return f"<d {self.name} {len(self.subdirs)} {len(self.files)}>"


class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size

    def __repr__(self):
        return f"<f {self.name} {self.size}> "


In [42]:
def load_fs(data):
    fs = wd = Directory("fs", parent=None)
    wd.add_subdir("/")
    for line in data.splitlines():
        tokens = line.split()
        try:
            a, b = tokens[:2]
        except IndexError:
            continue
        if a == "$" and b == "cd":
            wd = wd.cd(tokens[2])
        elif a == "dir":
            wd.add_subdir(b)
        elif a.isnumeric():
            wd.add_file(b, int(a))
    return fs


Compute each directory's tree size

In [87]:
%%time
# dfs
fs = load_fs(data)
stack = []
dirs = []
stack.append(fs)
while stack:
    d = stack.pop()
    dirs.append(d)
    for subd in d.subdirs.values():
        stack.append(subd)
for d in reversed(dirs):
    d.subtree_size += d.size
    if d.parent is not None:
        d.parent.subtree_size += d.subtree_size

CPU times: user 3.64 ms, sys: 0 ns, total: 3.64 ms
Wall time: 3.58 ms


Part 1

In [88]:
sum(d.subtree_size for d in dirs if d.subtree_size <= 100_000)

1844187

In [89]:
fs.subtree_size

43441553

In [90]:
total_space = 70_000_000
update_required_space = 30_000_000
current_free_space = total_space - fs.subtree_size
current_free_space

26558447

In [91]:
space_to_free = update_required_space - current_free_space
space_to_free

3441553

Part 2

In [92]:
min(d.subtree_size for d in dirs if d.subtree_size >= space_to_free)

4978279